# NCEI
<br>

- 월평균 활용
- https://apihub.kma.go.kr/
    - 세계기상 > NCEI 관측.통계 > 3. 전세계 지상관측(월통계) : 1763년~2022년
    - ※ NCEI(National Centers for Environmental Information, 국립환경정보센터)는 미국 NOAA의 소속된 기관으로 전세계 기상기후데이터를 수집·제공합니다.
    - ※ 모든 자료가 2022년 12월 까지 존재합니다.
## 참고 자료
- document 파일
    - 자료 소개서
        - GSOM_readme.txt
    - 자료 설명서
        - GSOM_documentation.pdf
    - 연/월 통합 상세 설명서
        - GSOM_GSOY_Description_Document_v1.0.2_20200219.pdf
- 지점정보
    - 링크 내용은 동일함
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.txt(내용확인)
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv(파일다운로드)
- 국가 코드
    - Country list
        - 국가명과 국가 2글자 표현 짝 정리된 것
        - 후에 요청인자 중 stn 구성요소인 지점코드값의 국가 부분을 해석하는데에 사용
        - 293개국 존재
- 파일 리스트
    - File list
        - 파일목록
        - 파일명과 파일 크기가 나열됨
        - 후술될 요청인자 중 stn의 구성요소인 지점코드 값으로 활용할 수 있음
            - 국가 2글자, 망구분 1글자, 지점번호 9글자를 가져올 소스
        - 122236개 있음
- 실제 파일 리스트
    - data_climate/gsom-latest 내부에 존재하는 파일들
    - 122041개 있음

## 코드 목적
- 사용 데이터: NCEI의 GSOM(전세계 월별 요약) 기후 데이터
- 데이터 위치: jihye 브랜치에 data_climate 폴더 내 gsom-latest csv 파일들
- 필요한 일 (우선순위 순서)
    1. Wine Data / OIV 폴더OIV_wine_producing_country_and_region.xlsx에 올라와있는 20개국(country_20)을 기준으로 데이터 정합성이 가장 좋은 해(year_1)를 고르기
        - 결과물
            1. year_1을 기준으로 country_20 데이터 누적한 파일
            2. year_1을 기준으로 나머지 국가 데이터 누적한 파일
    2. Wine Data / OIV 폴더OIV_wine_producing_country_and_region.xlsx에 올라와있는 20개국(country_20)을 기준으로 데이터 정합성이 가장 좋은 해 연속 5년(year_5)을 고르기
        - 결과물
            3. year_5을 기준으로 country_20 데이터 누적한 파일
            4. year_5을 기준으로 나머지 국가 데이터 누적한 파일
    3. 기후 데이터 EDA 아이디어 정리
    4. Wine Data / OIV 폴더OIV_wine_producing_country_and_region.xlsx에 올라와있는 20개국(country_20)을 기준으로 데이터 정합성이 가장 좋은 해 연속 10년(year_10)을 고르기
        - 결과물
            5. year_10을 기준으로 country_20 데이터 누적한 파일
            6. year_10을 기준으로 나머지 국가 데이터 누적한 파일
- 정합성 체크리스트
    - 12개월이 온전히 들어있는지
    - 결측치가 없는지
    - 가장 최신인지
    - 포도 재배 결과가 좋은 21, 20년도일 경우가 선호됨
- 20개국을 정합성 체크하는 기준으로 제한하는 이유
    - 모델 학습에 활용되므로 중요순위가 높다는 자체적 가중치를 둠
    - 역으로 기후 데이터에 정합성이 떨어지는 국가라면, 까다로운 품종 포도를 재배하는데에 적합한 정도의 기술력, 관심도가 떨어진다고 볼 수도 있음
    - 두 명이 주말 안에 못 함
    - 다만 20개국 정합성이 비슷한 두 해가 있고, 나머지 국가들의 데이터 정합성이 크게 차이난다면 차후 논의해볼 문제
- 활용 코드
    - climate_EDA.ipynb
        - open_csv('국가명') 함수를 이용하면 원하는 국가가 있는 모든 지역을 확인할 수 있음
        - open_csv_year('국가리스트', 연도) 함수를 이용하면 위 open_csv('국가명')을 활용해 불러온 국가가 있는 모든 지역 csv에서 특정 연도를 추출, 누적한 df를 얻을 수 있음

## 활용 library

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
dfFiletoCountry = pd.read_csv('data_climate/FiletoCountry.csv')
dfFiletoCountry

,RealFileNM,ID,COUNTRY NAME,Name
0,ACW00011604.csv,AC,ANTIGUA AND BARBUDA,ACW00011604
1,ACW00011647.csv,AC,ANTIGUA AND BARBUDA,ACW00011647
2,AE000041196.csv,AE,NaN,AE000041196
3,AEM00041194.csv,AE,NaN,AEM00041194
4,AEM00041217.csv,AE,NaN,AEM00041217
...,...,...,...,...
122036,ZI000067969.csv,ZI,ZIMBABWE,ZI000067969
122037,ZI000067975.csv,ZI,ZIMBABWE,ZI000067975
122038,ZI000067977.csv,ZI,ZIMBABWE,ZI000067977
122039,ZI000067983.csv,ZI,ZIMBABWE,ZI000067983


## 파일 목록 중에 활용할 csv를 불러오는 함수만들기
- 필요한 것
    - 적용 국가
        - Wine Data / OIV 폴더OIV_wine_producing_country_and_region.xlsx 파일 참고
    - 비적용 국가
        - 위 국가 뺀 나머지 국가들 정리

### 국가명 리스트 정리
- 적용국가
    - country_32
- 비적용 국가
    - countryNameList에서 country_32를 뺀 것

In [5]:
# 데이터 정합성 기준 국가
wineCountry = pd.read_excel('OIV_wine_producing_country_and_region.xlsx', engine = 'openpyxl')
# Null값은 제외해야 함
country_32 = list(wineCountry['Country'].dropna().unique()[:-1])
print(country_32.index('\xa0Japan'))
country_32[18] = 'Japan'

print(country_32)
print(len(country_32))

country_32 = [country.strip().upper() for country in country_32]

print(country_32)

C:\ProgramData\anaconda3\envs\ds_study\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


18
['Argentina', 'Australia', 'Austria', 'Belgium', 'Bulgaria', 'Brazil', 'Czech Republic', 'Switzerland', 'Chile', 'Croatia', 'Cyprus', 'Germany', 'Spain', 'France', 'Georgia', 'Greece', 'Hungary', 'Italy', 'Japan', 'Luxemburg', 'Morocco', 'Moldova', 'New Zealand', 'Peru', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Republic of Türkiye', 'Uruguay', 'South Africa']
32
['ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'BELGIUM', 'BULGARIA', 'BRAZIL', 'CZECH REPUBLIC', 'SWITZERLAND', 'CHILE', 'CROATIA', 'CYPRUS', 'GERMANY', 'SPAIN', 'FRANCE', 'GEORGIA', 'GREECE', 'HUNGARY', 'ITALY', 'JAPAN', 'LUXEMBURG', 'MOROCCO', 'MOLDOVA', 'NEW ZEALAND', 'PERU', 'PORTUGAL', 'ROMANIA', 'SERBIA', 'SLOVAKIA', 'SLOVENIA', 'REPUBLIC OF TÜRKIYE', 'URUGUAY', 'SOUTH AFRICA']


In [6]:
# NCEI 데이터 내 표기 국가명 변수
# Null값은 제외해야 함
countryNameList = list(dfFiletoCountry['COUNTRY NAME'].dropna().unique())
countryNameList = [country.strip() for country in countryNameList]
print(countryNameList)

['ANTIGUA AND BARBUDA', 'AFGHANISTAN', 'ALGERIA', 'AZERBAIJAN', 'ALBANIA', 'ARMENIA', 'ANGOLA', 'AMERICAN SAMOA', 'ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'ANTARCTICA', 'BAHRAIN', 'BARBADOS', 'BOTSWANA', 'BERMUDA', 'BELGIUM', 'BAHAMAS THE', 'BANGLADESH', 'BELIZE', 'BOSNIA AND HERZEGOVINA', 'BOLIVIA', 'BURMA', 'BENIN', 'BELARUS', 'SOLOMON ISLANDS', 'BRAZIL', 'BULGARIA', 'BRUNEI', 'BURUNDI', 'CANADA', 'CAMBODIA', 'CHAD', 'SRI LANKA', 'CONGO', 'ZAIRE', 'CHINA', 'CHILE', 'CAYMAN ISLANDS', 'COCOS (KEELING) ISLANDS', 'CAMEROON', 'COLOMBIA', 'NORTHERN MARIANA ISLANDS', 'COSTA RICA', 'CENTRAL AFRICAN REPUBLIC', 'CUBA', 'CAPE VERDE', 'COOK ISLANDS', 'CYPRUS', 'DENMARK', 'DOMINICA', 'DOMINICAN REPUBLIC', 'ECUADOR', 'EGYPT', 'IRELAND', 'EQUATORIAL GUINEA', 'ESTONIA', 'ERITREA', 'EL SALVADOR', 'ETHIOPIA', 'EUROPA ISLAND', 'CZECH REPUBLIC', 'FRENCH GUIANA', 'FINLAND', 'FIJI', 'FALKLAND ISLANDS (ISLAS MALVINAS)', 'MICRONESIA, FEDERATED STATES OF', 'FRENCH POLYNESIA', 'FRANCE', 'FRENCH SOUTHERN AND ANTAR

In [7]:
# NCEI 국가 리스트에서 기준 국가 빠진 리스트
a = len(countryNameList)
b = len(country_32)
# NCEI 국가 리스트에서 32개 뺀 값
print(a-b)

set_a = set(countryNameList)
set_b = set(country_32)

# b 차집합 a 결과
print(len(set_b.difference(set_a)))

# a 차집합 b 결과
print(set_a.difference(set_b))

# a 차집합 b 결과값에서 국가명 표기가 다른 곳 2군데를 찾음
# 룩셈부르크, 터키
# 추후에 작업할 때 참고해야 함
print('\n국가명 표기가 다른 곳')
print('LUXEMBURG' + ' > NCEI: ' + 'LUXEMBOURG')
print('REPUBLIC OF TÜRKIYE' + ' > NCEI: ' +  'TURKEY')
country_others = list(set_a.difference(set_b))
country_others.remove('LUXEMBOURG')
country_others.remove('TURKEY')

print(country_32.index('REPUBLIC OF TÜRKIYE'))
print(country_32.index('LUXEMBURG'))

country_32[29] = 'TURKEY'
country_32[19] = 'LUXEMBOURG'

len(country_others)

183
2
{'KUWAIT', 'MAYOTTE', 'COSTA RICA', 'MALDIVES', 'CENTRAL AFRICAN REPUBLIC', 'TRINIDAD AND TOBAGO', 'JUAN DE NOVA ISLAND', 'PITCAIRN ISLANDS', 'CAYMAN ISLANDS', 'IRAN', 'LESOTHO', 'WALLIS AND FUTUNA', 'ALGERIA', 'BURUNDI', 'PHILIPPINES', 'BAHRAIN', 'THAILAND', 'JAN MAYEN', 'SVALBARD', 'PAPUA NEW GUINEA', 'MACAU', 'CUBA', 'MARSHALL ISLANDS', 'GREENLAND', 'NIGER', 'PAKISTAN', 'DENMARK', 'KIRIBATI', 'SIERRA LEONE', 'TONGA', 'INDONESIA', 'COOK ISLANDS', 'BELIZE', 'EQUATORIAL GUINEA', 'VIRGIN ISLANDS (U.S.)', 'TURKMENISTAN', 'EL SALVADOR', 'BURKINA FASO', 'LATVIA', 'HONDURAS', 'NETHERLANDS', 'WESTERN SAHARA', 'GUINEA-BISSAU', 'RUSSIA', 'MONGOLIA', 'EUROPA ISLAND', 'CAMBODIA', 'NIGERIA', 'ERITREA', 'NORTHERN MARIANA ISLANDS', 'FRENCH GUIANA', 'KENYA', 'BOSNIA AND HERZEGOVINA', 'SINGAPORE', 'MONTENEGRO', 'NEPAL', 'BOLIVIA', 'TROMELIN ISLAND', 'TUNISIA', 'PARAGUAY', 'UKRAINE', 'JORDAN', 'BELARUS', 'DOMINICA', 'ECUADOR', 'MAURITIUS', 'GUINEA', 'NIUE', 'IRELAND', 'SYRIA', 'UNITED STATES', '

183

## 국가별 함수
- 국가별로 csv 파일을 불러와서 df_지점코드 라는 변수명에 저장한다
- df_지점코드 형식의 변수명은 varList라는 변수명 리스트에 저장된다
- varList가 반환된다.
- varList에 들어있는 i번째 변수명을 변수로 활용하는 방법
    - globals()[varList[i])

In [8]:
def open_csv(country = 'korea, south'):
    
    # 변수명 리스트
    varList = []
    
    # COUNTRY NAME에 country 인수가 포함되어있는 행을 추출
    csvList = dfFiletoCountry[dfFiletoCountry['COUNTRY NAME'].str.contains((country.upper())) == True]
    # 국가별 기상 관측 장소 갯수
    cnt = len(csvList)
    # 추출된 행의 RealFileNM값들을 리스트로 저장
    stnList = csvList['RealFileNM'].values
    
    # 
    for STN in stnList:
        varNM = 'df_{}'.format(STN)

        globals()[varNM] = pd.read_csv('data_climate/gsom-latest/' + STN)
        varList.append(varNM)
    
    print('변수 갯수: ', cnt)
          
    return varList

In [9]:
# 변수명은 상관X
koreaList = open_csv()

변수 갯수:  56


## 특정 연도 데이터 추출 및 누적 함수
- 변수명이 들어있는 리스트를 활용해 특정연도 데이터를 추출하여 누적
- dfTMP에 활용 df를 순차적으로 할당한다.
- or_args(\*args)함수를 활용해 복수의 연도를 적용한다.
    - 이때 연도는 문자타입으로 나열함
    - ex) or_args(varList, '2022', '2021', ...)
- df변수에 dfTMP에서 특정연도 추출한 행을 누적한다.
- df를 반환한다.

In [10]:
def open_csv_year(varList, *year):
    
    flag = False
    
    # 국가별 df 변수를 순서대로 불러 df_TMP에 저장
    for i in range(len(varList)):
        
#         print('적용 df: ', varList[i])
        dfTMP = globals()[varList[i]]
        
        # 지정된 연도를 |로 묶어서 str 변수를 반환
        def or_args(*args):
            items = ''

            for num, item in enumerate(args):
                if num == len(args) - 1:
                    items += item
                else:
                    items += item + '|'

            return items

        # 활용 연도
        Years = or_args(*year)
        tmp = dfTMP[dfTMP['DATE'].str.contains(Years)]
        # csv 처음 불러왔을 때
        if flag == False:
#             print('0')
            # 결과 df 변수에 Years가 포함된 데이터만 추출
            df = tmp
            # flag를 변경하여 이 if문에 다시 들어오지 않는다.
            flag = True
            print('길이: ', len(df))
        else:
#             print('1')
            # 결과 df와 지금 df의 Years가 포함된 데이터를 이어붙인다.
            df = pd.concat([df, tmp], axis = 0)
            print('추가 행 길이: ', len(tmp))
            
    df.reset_index(inplace = True, drop = True)
    print(Years, str(varList))
    
    return df

In [11]:
dfKorea = open_csv_year(koreaList, '2022', '2021')

길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  4
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
2022|2021 ['df_KS000047112.csv', 'df_KS000047115.csv', 'df_KS000047165.csv', 'df_KSM00047101.csv', 'df_KSM00047105.csv', 'df_KSM00047108.csv', 'df_KSM00047133.csv', 'df_KSM00047138.csv', 'df_KSM00047159.csv', 'df_KSM00047168.csv', 'df_KSM00047184.csv', 'df_KSW00043201.csv', 'df_KSW00043206.csv', 'df_KSW00043207.csv', 'df_KSW0004320

In [12]:
dfKorea.head(50)

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,CDSD,CDSD_ATTRIBUTES,CLDD,CLDD_ATTRIBUTES,...,DYTS,DYTS_ATTRIBUTES,EMSN,EMSN_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,WDFG,WDFG_ATTRIBUTES,WSFG,WSFG_ATTRIBUTES
0,KS000047112,2022-05,37.467,126.633,70.0,"INCHEON, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KS000047112,2022-06,37.467,126.633,70.0,"INCHEON, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KS000047112,2022-08,37.467,126.633,70.0,"INCHEON, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KS000047115,2022-05,37.483,130.900,220.0,"ULLEUNGDO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KS000047115,2022-06,37.483,130.900,220.0,"ULLEUNGDO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,KS000047115,2022-08,37.483,130.900,220.0,"ULLEUNGDO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,KS000047165,2022-05,34.817,126.383,39.0,"MOKPO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,KS000047165,2022-06,34.817,126.383,39.0,"MOKPO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,KS000047165,2022-08,34.817,126.383,39.0,"MOKPO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,KSM00047101,2022-05,37.900,127.733,78.0,"CHUNCHEON, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# !!! 현정님 이곳입니다!!!
- 기준 국가_32를 적용한 데이터의 특정 연도 데이터 확인하기
- 나머지 국가에 특정 연도 데이터 확인하기

In [13]:
# 기준 국가_32 변수
print(country_32)

# NCEI 데이터 내 표기 국가명 변수
print(countryNameList)

# 기준 국가 빠진 나머지 국가
# 정확한 코드 아님, 직관적 이해 위해 적은 것
# country_others = countryNameList - country_20
print(country_others)

['ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'BELGIUM', 'BULGARIA', 'BRAZIL', 'CZECH REPUBLIC', 'SWITZERLAND', 'CHILE', 'CROATIA', 'CYPRUS', 'GERMANY', 'SPAIN', 'FRANCE', 'GEORGIA', 'GREECE', 'HUNGARY', 'ITALY', 'JAPAN', 'LUXEMBOURG', 'MOROCCO', 'MOLDOVA', 'NEW ZEALAND', 'PERU', 'PORTUGAL', 'ROMANIA', 'SERBIA', 'SLOVAKIA', 'SLOVENIA', 'TURKEY', 'URUGUAY', 'SOUTH AFRICA']
['ANTIGUA AND BARBUDA', 'AFGHANISTAN', 'ALGERIA', 'AZERBAIJAN', 'ALBANIA', 'ARMENIA', 'ANGOLA', 'AMERICAN SAMOA', 'ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'ANTARCTICA', 'BAHRAIN', 'BARBADOS', 'BOTSWANA', 'BERMUDA', 'BELGIUM', 'BAHAMAS THE', 'BANGLADESH', 'BELIZE', 'BOSNIA AND HERZEGOVINA', 'BOLIVIA', 'BURMA', 'BENIN', 'BELARUS', 'SOLOMON ISLANDS', 'BRAZIL', 'BULGARIA', 'BRUNEI', 'BURUNDI', 'CANADA', 'CAMBODIA', 'CHAD', 'SRI LANKA', 'CONGO', 'ZAIRE', 'CHINA', 'CHILE', 'CAYMAN ISLANDS', 'COCOS (KEELING) ISLANDS', 'CAMEROON', 'COLOMBIA', 'NORTHERN MARIANA ISLANDS', 'COSTA RICA', 'CENTRAL AFRICAN REPUBLIC', 'CUBA', 'CAPE VERDE', 'COO

In [14]:
# country_32에 있는 국가명을 다음 함수에 넣어주세요
A = open_csv('korea')
ㄴ
# 위 함수 결과물과 원하는 연도를 아래 함수에 넣어주세요
B = open_csv_year(A, '2022')

변수 갯수:  66
길이:  4
추가 행 길이:  4
추가 행 길이:  4
추가 행 길이:  4
추가 행 길이:  4
추가 행 길이:  4
추가 행 길이:  4
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
2022 ['df_KNM00047014.csv', 'df_KNM00047016.csv', 'df_KNM00047025.csv', 'df_KNM00047035.csv', 'df_KNM00047055.csv', 'df_KNM00047058.csv', 'df_KNM00047069.csv', 'df_KNW00043225.csv', 'df_KNW00043241.csv'

In [15]:
# 특정 연도의 특정 국가의 데이터 정합성을 편하신 방법으로 확인부탁드립니다.
B

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,CDSD,CDSD_ATTRIBUTES,CLDD,CLDD_ATTRIBUTES,...,DYTS,DYTS_ATTRIBUTES,EMSN,EMSN_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,WDFG,WDFG_ATTRIBUTES,WSFG,WSFG_ATTRIBUTES
0,KNM00047014,2022-05,41.783,126.883,331.0,"CHUNGGANG, KN",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KNM00047014,2022-06,41.783,126.883,331.0,"CHUNGGANG, KN",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KNM00047014,2022-07,41.783,126.883,331.0,"CHUNGGANG, KN",NaN,NaN,170.1,"5,S",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KNM00047014,2022-08,41.783,126.883,331.0,"CHUNGGANG, KN",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KNM00047016,2022-05,41.400,128.167,714.0,"HYESAN, KN",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,KSM00047168,2022-06,34.733,127.750,67.0,"YEOSU, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,KSM00047168,2022-08,34.733,127.750,67.0,"YEOSU, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,KSM00047184,2022-05,33.517,126.533,23.0,"JEJU, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,KSM00047184,2022-06,33.517,126.533,23.0,"JEJU, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 이 아래는 디버깅 중
- 위에서 편하게 작업해주세요

In [ ]:
# 특정 국가, 연도가 적용된 결과 df 모아둔 것
# 정확한 코드 아님, 직관적 이해 적은 것
def dfCountryYear(countryList, *years):
    flag = False

    for country in countryList:
        
        print('현재 나라: ', country)
        
        # 특정 국가의 모든 구역 파일명 리스트
        tmpList = open_csv(country)
        
        # 특정 국가의 모든 구역 파일명 리스트와 연도가 적용된 df
        tmp = open_csv_year(tmpList, *years)
    
        # csv 처음 불러왔을 때
        if flag == False:
            print('0')
            # 결과 df 변수에 첫 번째 국가 df를 적용
            df = tmp
            # flag를 변경하여 이 if문에 다시 들어오지 않는다.
            flag = True
            print('df 길이: ', len(df))
        else:
            print('1')
            # 결과 df와 현재 국가 df를 이어붙임
            df = pd.concat([df, tmp], axis = 0)
            print('df 추가 길이: ', len(tmp))
            
    return df

In [ ]:
# 기준 국가
dfResult = dfCountryYear(country_32[1:2], '2022')

len(dfResult)
dfResult

In [ ]:
dfResult

In [ ]:
# 나머지 국가
dfResult = dfCountryYear(country_others)

len(dfResult)
dfResult

In [ ]:
# 추후에 결정되면 저장할때 활용
dfResult.to_csv('data_climate/' + + '.csv', index = False)

# ! 회의 이후에 필요한 내용 추가하여 시각화 해야함
## EDA


### 시각화 참고 자료
- plt.hist
    - https://lifelong-education-dr-kim.tistory.com/entry/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%9D%98-%EB%B6%84%ED%8F%AC%EB%A5%BC-%EA%B0%80%EC%8B%9C%ED%99%94-%ED%95%98%EB%8A%94-matplotlib-%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC%EC%9D%98-hist

In [ ]:
dfTMP = globals()[koreaList[0]]
dfTMP.info()

In [ ]:
plt.figure(figsize = (10, 8))

dfTMP['TMAX'].hist(color = 'r', alpha = 0.5)
dfTMP['TMIN'].hist(color = 'b', alpha = 0.5)
dfTMP['TAVG'].hist(color = 'g', alpha = 0.5)
plt.show()